In [ ]:
!pip install pydriller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.3 MB/s eta 0:00:00


Considered filters by choosing python, then restricting commits between 1000 and 10000, with populairty stars between 10,000 and 20,000 with key word data.

This returned 5 repos, out of which the below repo has been chosen.

In [ ]:
data_url="https://github.com/simonw/datasette"
!git clone https://github.com/simonw/datasette


Cloning into 'datasette'...
remote: Enumerating objects: 19693, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 19693 (delta 156), reused 120 (delta 119), pack-reused 19490 (from 3)
Receiving objects: 100% (19693/19693), 7.27 MiB | 16.26 MiB/s, done.
Resolving deltas: 100% (15055/15055), done.


In [ ]:
from git import Repo
r = Repo("/content/datasette")
all_commits = list(r.iter_commits())

In [ ]:
%cd datasette

/content/datasette


In [ ]:
print(all_commits)
print(len(all_commits))

[<git.Commit "7a602140df3646820adc45963daf7fc5dcd2a009">, <git.Commit "e2497fdb595055a63f2c9b7b522e76d501be224c">, <git.Commit "1c77a7e33f7902d3115ace6fbd6297391b9bc640">, <git.Commit "6f7f4c7d89b37187667441ce9df583f6dbbe2977">, <git.Commit "f4274e7a2e22660c02b205aae3a812a9bc281bf2">, <git.Commit "271aa09056eb4f7cbfafb8c5f2e8df4c103ff413">, <git.Commit "d5c6e502fbdaacd9143ce160c319066488b5d6be">, <git.Commit "f2485dce9cf5644d8c35cb697257d055a2b32bc2">, <git.Commit "f6446b3095ee8c4f3dae405f5295713860cf2e3c">, <git.Commit "d03273e205492ea16829303b798fc7c354368254">, <git.Commit "d021ce97aa60c48441bad7a976112b2bb11f6ccd">, <git.Commit "7945f4fbf2fac201f6aac8f5a84c10e0777fea82">, <git.Commit "da209ed2bafd101e92fe75c9da68093626ce93a8">, <git.Commit "333f786cb0b0255d6afdc50364a025646cf1e8e6">, <git.Commit "6e512caa597da610671ee5c696c9b65892eabb56">, <git.Commit "209bdee0e8d1b578c48fe6dbc2e51739dc1d1f0a">, <git.Commit "e59fd0175708f2b14d4e3c08ea16631bda0aaed3">, <git.Commit "cd9182a5511d2a243

In [ ]:
from pydriller import Repository
import pandas as pd
words = [
    "fixed", "bug", "fixes", "fix", "crash",
    "solves", "resolves", "issue", "regression",
    "fall back", "assertion", "coverity", "reproducible",
    "stack-wanted", "steps-wanted", "testcase", "failur", "fail",
    "npe", "except", "broken", "differential testing", "error",
    "hang", "test fix", "steps to reproduce", "failure", "leak",
    "stack trace", "heap overflow", "freez", "problem",
    "overflow", "avoid", "workaround", "break", "stop"
]


In [ ]:
def is_bugfix(msg):
  if msg is None:
    return False
  msg = msg.lower()
  return any(k in msg for k in words)

In [ ]:
rows = []
for commit in Repository('/content/datasette').traverse_commits():
  if is_bugfix(commit.msg):
    modified_files= [m.new_path or m.old_path for m in commit.modified_files]
    parent_hashes= ";".join(p for p in commit.parents)
    rows.append({
        "Hash": commit.hash,
        "Message": commit.msg,
        "Hashes of parents": parent_hashes,
        "Is it merge": len(commit.parents) > 1,
        "Modified files list": ";".join(modified_files)
        })

df = pd.DataFrame(rows)
df.to_csv("bugged_commits.csv", index=False)

In [ ]:
df= pd.read_csv("bugged_commits.csv")
print(df.info())
print(len(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Hash                 725 non-null    object
 1   Message              725 non-null    object
 2   Hashes of parents    725 non-null    object
 3   Is it merge          725 non-null    bool  
 4   Modified files list  725 non-null    object
dtypes: bool(1), object(4)
memory usage: 23.5+ KB
None
725


In [ ]:
df = pd.read_csv("bugged_commits.csv")
df['Message'] = df['Message'].str.slice(0, 80) + '...'
print(df.head(10).to_string(index=False))

                                    Hash                                                                               Message                        Hashes of parents  Is it merge                                                                          Modified files list
f571b19d8a5fd3a19fdf679421fd55a8edbf5295            sqlerrors() decorator catching and returning useful errors\n\nCloses #8... de04d7a854d71003ffcf98028eab976a936c2dba        False                                                                                       app.py
6a0c5de6154893eb4269dc7b1f160726ec6395f2 ensure_build_metadata() function for metadata\n\nThis will be run at compile time ... f571b19d8a5fd3a19fdf679421fd55a8edbf5295        False                                                       .gitignore;app.py;templates/index.html
b2372605d63248f422b6e67cb5c392236a3aa612   Implemented multi-db support plus initial URL structure\n\nRefs #24\n\nFixes #15... 6a0c5de6154893eb4269dc7b1f160726ec6395f2        Fal

In [ ]:
rows = []
bugged_df = pd.read_csv("bugged_commits.csv")
bugged_hashes = set(bugged_df["Hash"].dropna().unique())

for commit in Repository('/content/datasette').traverse_commits():
  if commit.hash not in bugged_hashes:
    continue
  for m in commit.modified_files:
    rows.append({
        "Hash": commit.hash,
        "Message": commit.msg,
        "Filename": m.new_path or m.old_path,
        "Source code before": m.source_code_before or "",
        "Source code current": m.source_code or "",
        "Diff": m.diff or "",
        })

diffs = pd.DataFrame(rows)
diffs.to_csv("file_diffs_no_llm.csv", index=False)

In [ ]:
df= pd.read_csv("file_diffs_no_llm.csv")
print(df.info())
print(len(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710 entries, 0 to 1709
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Hash                 1710 non-null   object
 1   Message              1710 non-null   object
 2   Filename             1710 non-null   object
 3   Source code before   1657 non-null   object
 4   Source code current  1697 non-null   object
 5   Diff                 1708 non-null   object
dtypes: object(6)
memory usage: 80.3+ KB
None
1710


In [ ]:
print("Unique commits in file-level diff data:", df["Hash"].nunique())
print("Total rows (files):", len(df))

Unique commits in file-level diff data: 725
Total rows (files): 1710


In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 132.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5")
model = AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
rows=[]
df = pd.read_csv("file_diffs_no_llm.csv")

def llm_inference(diff_text):
  if not isinstance(diff_text, str) or diff_text.strip() == "":
    return ""
  inputs = tokenizer(diff_text, return_tensors="pt", truncation=True, max_length=512)
  outputs = model.generate(**inputs, max_length=50, num_beams=5, early_stopping=True)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)


df["llm_inference"] = df["Diff"].apply(llm_inference)

df.to_csv("file_diffs_with_llm.csv", index=False)

In [ ]:
df = pd.read_csv("file_diffs_with_llm.csv")
print(df.head())
print(df.info())
print(len(df))

                                       hash  \
0  f571b19d8a5fd3a19fdf679421fd55a8edbf5295   
1  6a0c5de6154893eb4269dc7b1f160726ec6395f2   
2  6a0c5de6154893eb4269dc7b1f160726ec6395f2   
3  6a0c5de6154893eb4269dc7b1f160726ec6395f2   
4  b2372605d63248f422b6e67cb5c392236a3aa612   

                                             message              filename  \
0  sqlerrors() decorator catching and returning u...                app.py   
1  ensure_build_metadata() function for metadata\...            .gitignore   
2  ensure_build_metadata() function for metadata\...                app.py   
3  ensure_build_metadata() function for metadata\...  templates/index.html   
4  Implemented multi-db support plus initial URL ...            .gitignore   

                                                diff  is_merge  \
0  @@ -2,6 +2,7 @@ from sanic import Sanic\n from...     False   
1  @@ -1,3 +1,5 @@\n+build-metadata.json\n+\n # B...     False   
2  @@ -2,8 +2,55 @@ from sanic import Sanic\n fro.

In [25]:
!git clone https://github.com/vyomika363/stt_for_cse_lab2

fatal: destination path 'stt_for_cse_lab2' already exists and is not an empty directory.


In [ ]:
!cp /content/stt_for_cse_lab2.ipynb /content/stt_for_cse_lab2/

cp: cannot stat '/content/stt_for_cse_lab2.ipynb': No such file or directory


In [ ]:
%cd /content/stt_for_cse_lab2

/content/stt_for_cse_lab2


In [ ]:
!git config --global user.email "23110363@iitgn.ac.in"
!git config --global user.name "Vyomika Vasireddy"

In [26]:
!git add .
!git commit -m "Adding file upto llm inference part"
!git push origin main

error: 'stt_for_cse_lab2/' does not have a commit checked out
fatal: adding files failed
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	stt_for_cse_lab2/

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/vyomika363/stt_for_cse_lab2'


In [27]:
!ls

stt_for_cse_lab2
